In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score , recall_score, f1_score
import time

In [2]:
diabetes_df = pd.read_csv("diabetes_data_upload.csv")
column_names = diabetes_df.columns.values
diabetes_df

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,39,Female,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Yes,Yes,No,No,No,Positive
516,48,Female,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,No,No,No,Positive
517,58,Female,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,Yes,No,Yes,Positive
518,32,Female,No,No,No,Yes,No,No,Yes,Yes,No,Yes,No,No,Yes,No,Negative


In [3]:
class Node():
    def __init__(self, feature_index=None, feature_type=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.feature_type = feature_type
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

In [4]:
class DecisionTreeClassifier():
    
    
    def __init__(self, max_depth = 5):
        self.root = None
        self.max_depth = max_depth


    def build_tree(self, dataset, curr_depth=0):
        
        isPure = self.is_pure(dataset)
        
        if (curr_depth <= self.max_depth) and (isPure != True):
            best_split = self.get_best_split(dataset)
            if (best_split["info_gain"] > 0):
                 # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["feature_type"] ,best_split["threshold"], left_subtree, right_subtree, best_split["info_gain"])


        values,counts = np.unique(dataset[:, -1], return_counts=True)

        leaf_index = np.where(counts == max(counts))
        leaf_value = values[leaf_index]

        return Node(value=leaf_value)


    def get_best_split(self,dataset):
        best_split = {}
        info_gains = list()
        
        for feature_index in range(len(dataset[0])-1): #Using this to loop over the features. It doesn't matter what # is in dataset[#].
            feature = dataset[:, feature_index]
            thresholds = np.unique(feature) #thresholds for continuous
            feature_type = self.get_featureType(dataset, feature_index)
            
            if(feature_type == 'Continuous'):
                for threshold in thresholds:
                    left, right = self.split_cont(dataset, feature_index, threshold)
                    if (len(left) > 0 and  len(right) > 0):
                        info_gain = self.Information_gain(dataset, left, right)
                        info_gains.append([feature_index,feature_type,threshold, left, right, info_gain])
                continue
            else:
                if('Yes' in thresholds):
                    left, right = self.split_disc(dataset, feature_index, 'Yes')
                    if (len(left) > 0 and  len(right) > 0):
                        info_gain = self.Information_gain(dataset, left, right)
                        info_gains.append([feature_index, feature_type, 'Yes', left, right, info_gain])
                else: #It is gender 
                    left, right = self.split_disc(dataset, feature_index, 'Male')
                    if (len(left) > 0 and  len(right) > 0):
                        info_gain = self.Information_gain(dataset, left, right)
                        info_gains.append([feature_index, feature_type,'Male', left, right, info_gain])
                continue
            
        info_gains.sort(key = lambda x : x[-1], reverse=True)
        
        if(len(info_gains) == 0):
            print(dataset)
        
        best_split["feature_index"] = info_gains[0][0]
        best_split["feature_type"] = info_gains[0][1]
        best_split["threshold"] = info_gains[0][2]
        best_split["dataset_left"] = info_gains[0][3]
        best_split["dataset_right"] = info_gains[0][4]
        best_split["info_gain"] = info_gains[0][5]
            
        return best_split

    def is_pure(self,dataset):
        values = np.unique(dataset[:, -1])
        if(len(values) == 1):
            return True
        else:
            return False

    def split_cont(self, dataset, feature_index, threshold):

        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right

    def split_disc(self, dataset, feature_index, threshold):

        dataset_left = np.array([row for row in dataset if row[feature_index]==threshold]) #Yes, Male
        dataset_right = np.array([row for row in dataset if row[feature_index]!=threshold]) #No, Female
        return dataset_left, dataset_right


    def entropy(self, dataset): 
        
        count_positive = 0
        count_negative = 0
        
        for i in dataset:
            if i[-1] == 'Positive':
                count_positive += 1
            else:
                count_negative += 1
                
        entropy = 0
                
        if(count_positive != 0):
            entropy += -(count_positive/len(dataset)) * math.log2(count_positive/len(dataset))
        if(count_negative != 0):
            entropy += -(count_negative/len(dataset)) * math.log2(count_negative/len(dataset))
        
        return entropy
        

    def Information_gain(self, parent, left_child, right_child):

        probs_l = len(left_child)/len(parent)
        probs_r = len(right_child)/len(parent)

        #information gain = Entropy(parent) - Entropy(Childs)
        information_gain = self.entropy(parent) - probs_l*self.entropy(left_child) - probs_r*self.entropy(right_child)
        return information_gain

    def get_featureType(self,dataset, feature_index):
        feature = dataset[:, feature_index]
        values = np.unique(feature)
        if(len(values) > 2):  #In our dataset discrete features only have 2 unique values.(Male-Female, Yes-No)
            return 'Continuous'
        else :
            return 'Discrete'
            
    def fit(self, X, Y):
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions
    
    def make_prediction(self, x, tree):
        
        if tree.value!=None: 
            return tree.value
        
        feature_val = x[tree.feature_index]
        feature_type = tree.feature_type
        if(feature_type == 'Continuous'):
            if feature_val<=tree.threshold:
                return self.make_prediction(x, tree.left)
            else:
                return self.make_prediction(x, tree.right) 
        else:
            if feature_val==tree.threshold:
                return self.make_prediction(x, tree.left)
            else:
                return self.make_prediction(x, tree.right)
            
    
    ###############################################PRUNE OPERATIONS############################################################
            
    def twig_check(self, given_node):
        if given_node.right is not None and given_node.left is not None: # Check if node is not a leaf
            if given_node.right.value is not None and given_node.left.value is not None: # check if children are not leaf
                return True
        return False
    
    def detect_min_recursive(self, current_node, infoGain_list):
        if self.twig_check(current_node): # if twig then append its info_gain into the list
            infoGain_list.append(current_node.info_gain)
        else:
            if current_node.left is not None: # check if children are not leaf
                self.detect_min_recursive(current_node.left, infoGain_list)
            if current_node.right is not None:# check if children are not leaf
                self.detect_min_recursive(current_node.right, infoGain_list)
    
    def detect_min_infoGain(self):
        twig_infoGains = list() # create info_gain list
        self.detect_min_recursive(self.root, twig_infoGains) # list is filled with detect_min_recursive function
        min_infoGain = min(twig_infoGains) # find min info_gain in list
        return min_infoGain # return it
        
    def find_majority(self, road_data):
        diabetes_df_copy = diabetes_df.copy()
        diabetes_arr = diabetes_df.to_numpy()
        
        indexes = list() # the list which holds indexes to of dataset to be considered, empty fow now
        
        for i in road_data: # road data holds the feature indexes and their values which should be considered
            for c in range(len(diabetes_arr)):
                if(i[0]!=0): #  if not Age column
                    if(diabetes_arr[c][i[0]] == i[1]): # if values is same
                        indexes.append(c) # append to list the index of row
                else:
                    if(i[1] == 'Yes' and diabetes_arr[c][i[0]] < i[2]): # Column is age check if value is smaller than threshold
                        indexes.append(c)
                    elif(i[1] == 'No' and diabetes_arr[c][i[0]] > i[2]): # if not smaller than threshold, append No
                        indexes.append(c)
        ## indexes has been done
        
        positive_count = 0
        negative_count = 0
        
        for i in indexes: # now we have indexes
            for z in range(len(diabetes_arr)):
                if i == z:
                    if(diabetes_arr[z][-1] == 'Positive'):
                        positive_count += 1
                    elif(diabetes_arr[z][-1] == 'Negative'):
                        negative_count += 1
                    else:
                        print('Neither negative nor positive')
                    
        if(positive_count>=negative_count): # detect majority
            return 'Positive'
        else:
            return 'Negative'
        
    def getNode_recursive(self, given_node, infoGain, last_accuracy, valData_x, valData_y, return_data, road_data):
        
        
        if(given_node.info_gain == infoGain): # check if we've found the node we are searching for
            
            majority = self.find_majority(road_data) #  define majority(Positive or Negative)
            majority_array = np.array([majority])
            
            node_data = list() # Save data of node in case of revert
            node_data.append(given_node.feature_index)
            node_data.append(given_node.feature_type)
            node_data.append(given_node.threshold)
            node_data.append(given_node.left)
            node_data.append(given_node.right)
            node_data.append(given_node.info_gain)
            
            given_node.feature_index = None # Set the twig to a Leaf with value majority
            given_node.feature_type = None
            given_node.threshold = None
            given_node.left = None
            given_node.right = None
            given_node.info_gain = None
            given_node.value = majority_array
            
            new_predictions = self.predict(valData_x) # predict again
            new_accuracy = accuracy_score(valData_y, new_predictions) # get new accuracy
            
            print('last accuracy: ' + str(last_accuracy) + ' //// current accuracy: ' + str(new_accuracy))
            
            if(new_accuracy > last_accuracy):
                return_data.append(True) # This will be used to decide whether contuinue to loop or break
                return_data.append(new_accuracy)
                
            else: #  Revert case, set leaf to the twig by using saved data
                given_node.feature_index = node_data[0]
                given_node.feature_type = node_data[1]
                given_node.threshold = node_data[2]
                given_node.left = node_data[3]
                given_node.right = node_data[4]
                given_node.info_gain = node_data[5]
                given_node.value = None
                
                return_data.append(False) # means break the loop

        else: # if not a twig
            
            road_data_left = road_data.copy() # copy road for left child
            road_data_right = road_data.copy() # copy road for right child 
            
            if(given_node.feature_index==0): # if column is age
                temp_list = list()
                temp_list.append(0)
                temp_list.append('Yes')# add yes to road_data for right child
                temp_list.append(given_node.threshold) # we also need to append treshold to road_data
                road_data_left.append(temp_list) # append datas to road_data of left child
                
                temp_list_right = list()
                temp_list_right.append(0)
                temp_list_right.append('No')# add No to road_data for right child
                temp_list_right.append(given_node.threshold) # we also need to append treshold to road_data
                road_data_right.append(temp_list_right) # append datas to road_data of right child
                
            else: # if column is not age
                temp_list = list()
                temp_list.append(given_node.feature_index) # simply add feature index to road_data
                temp_list.append('Yes') # add No to road_data for left child
                
                road_data_left.append(temp_list) # append datas to road_data of left child
                
                temp_list_right = list()
                temp_list_right.append(given_node.feature_index)# simply add feature index to road_data
                temp_list_right.append('No') # add No to road_data for right child
                
                road_data_right.append(temp_list_right) # append datas to road_data of right child
            
            if given_node.left is not None: # if child is exist
                if given_node.left.value is None: # if child is not a leaf call recursive method on it
                    self.getNode_recursive(given_node.left, infoGain, last_accuracy, valData_x, valData_y,return_data, road_data_left)
            if given_node.right is not None: # if child is exist
                if given_node.right.value is None: # if child is not a leaf call recursive method on it
                    self.getNode_recursive(given_node.right,infoGain, last_accuracy, valData_x, valData_y,return_data, road_data_right)
                    
                    
    def prune(self, valData_x, valData_y):
        valData_predict = self.predict(valData_x)
        last_accuracy = accuracy_score(valData_y, valData_predict) ## initialize last accuracy
        
        
        while True:
            return_data = list() # recursive methods fill this with true or false so we decide to break loop or not
            min_infoGain = self.detect_min_infoGain()
            road_data_initial = list() # road data will be filled by recursive method and will be used for Rules
            self.getNode_recursive(self.root, min_infoGain, last_accuracy, valData_x, valData_y, return_data, road_data_initial)
            if not return_data[0]: # if false means new accuracy is smaller so break
                break
            
    
    def feature_by_index(self, feature_index): # get feature index return related column name
        columns_list = diabetes_df.columns.values
        return columns_list[feature_index]
    
    
    def detect_ways(self, current_node, single_road, the_map):
        if current_node.value is not None: # means if current node is a leaf
            single_road.append(current_node.value) # append road the value of leaf POSITIVE OR NEGATIVE
            the_map.append(single_road) # APPEND data of whole way to map list
        else:
            single_road.append(self.feature_by_index(current_node.feature_index)) # append feature index to way list
            if(current_node.feature_index == 0):
                single_road.append(current_node.threshold) # if node is Age, append threshold to way list
            left_list = single_road.copy() # copy way list for left child
            right_list = single_road.copy() # copy way list for right child
            if current_node.left is not None: # if left child is exist
                left_list.append('Yes') # append yes for left child to way list
                self.detect_ways(current_node.left, left_list, the_map) # call method on left child
            if current_node.right is not None: # if right child is exist
                right_list.append('No') # append no for right child to way list
                self.detect_ways(current_node.right, right_list, the_map)# call method on right child
                
    def print_rules(self, return_data): # Since this method is only for print a given list, making comment on this is not necessary
        single_road = list()
        the_map = list()
        
        self.detect_ways(self.root, single_road, the_map)
        
        for i in the_map:# yes ve noysa bosluk degilse = koy 
            road_string = ''
            for z in range(len(i)):
                if (i[z]=='Yes' or i[z]=='No'):
                    road_string += str(i[z])
                    if(z != len(i)-2):
                        road_string += '^'
                    else:
                        road_string += ' => '
                else:
                    if(i[z] == 'Age'):
                        road_string += 'Age<'
                    else:
                        road_string += str(i[z])
                        if(z != len(i)-1):
                            road_string += '='
                            
            return_data.append(road_string)
    
def print_givenList(given_list):
    for i in given_list:
        print(i)

In [5]:
classifier = DecisionTreeClassifier(max_depth=50)

In [6]:

X = diabetes_df.iloc[:, :-1].values
y = diabetes_df.iloc[:, -1].values.reshape(-1,1)
kf = KFold(n_splits=5, shuffle=True, random_state=1)
kf.get_n_splits(X)

fold = 1
rules_list = list()
start_time = time.time()

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    classifier.fit(X_train,y_train)
    Y_pred = classifier.predict(X_test) 
    print('For fold ' + str(fold))
    print('Accuracy: ' + str(accuracy_score(y_test, Y_pred)))
    print('Precision: ' + str(precision_score(y_test, Y_pred, average = None, zero_division = 0)[1]))
    print('Recall: ' + str(recall_score(y_test, Y_pred, average = None, zero_division=0)[1]))
    print('f1: ' + str(f1_score(y_test, Y_pred, pos_label='Positive')))
    print('\n')
    if fold==4:
        classifier.print_rules(rules_list)
    fold+=1
end_time = time.time()
print('\n Computation time: ' + str(end_time - start_time))

print('\n\n Misclassified samples from Fold 5\n')

for i in range(len(Y_pred)):
    if(i == 6 or i == 27):
        print(str(X_test[i]) + '\nExpected: ' +str(y_test[i][0]) + ' Predicted: ' + str(Y_pred[i][0]) + '\n')

For fold 1
Accuracy: 0.9519230769230769
Precision: 0.9552238805970149
Recall: 0.9696969696969697
f1: 0.9624060150375939


For fold 2
Accuracy: 0.9807692307692307
Precision: 0.9701492537313433
Recall: 1.0
f1: 0.9848484848484849


For fold 3
Accuracy: 0.9423076923076923
Precision: 0.9661016949152542
Recall: 0.9344262295081968
f1: 0.95


For fold 4
Accuracy: 0.9903846153846154
Precision: 1.0
Recall: 0.9855072463768116
f1: 0.9927007299270074


For fold 5
Accuracy: 0.9230769230769231
Precision: 0.9811320754716981
Recall: 0.8813559322033898
f1: 0.9285714285714285



 Computation time: 0.9967048168182373


 Misclassified samples from Fold 5

[42 'Male' 'No' 'No' 'No' 'Yes' 'Yes' 'No' 'No' 'No' 'Yes' 'No' 'No' 'Yes'
 'No' 'No']
Expected: Positive Predicted: Negative

[54 'Male' 'No' 'No' 'No' 'No' 'No' 'Yes' 'No' 'Yes' 'Yes' 'No' 'No' 'Yes'
 'Yes' 'Yes']
Expected: Positive Predicted: Negative



##  Part2

In [7]:

X_train, X_test, Y_train, Y_test = train_test_split(diabetes_df.iloc[:, :-1], diabetes_df.iloc[:, -1], test_size = 0.20, random_state=24)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.25, random_state=13)

Y_train = Y_train.values.reshape(-1,1)
Y_test = Y_test.values.reshape(-1,1)
Y_val = Y_val.values.reshape(-1,1)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
X_val = X_val.to_numpy()


classifier.fit(X_train, Y_train)
Y_pred = classifier.predict(X_test)

print('Pre-prune: ')
print('Accuracy: ' + str(accuracy_score(y_test, Y_pred)))
print('Precision: ' + str(precision_score(y_test, Y_pred, average = None, zero_division = 0)[1]))
print('Recall: ' + str(recall_score(y_test, Y_pred, average = None, zero_division=0)[1]))
print('f1: ' + str(f1_score(y_test, Y_pred, pos_label='Positive')))
print('\n')

print('Prune process:\n')

rulesList_prePrune = list()
classifier.print_rules(rulesList_prePrune)

classifier.prune(X_val, Y_val)
Y_pred = classifier.predict(X_test)

rulesList_postPrune = list()
classifier.print_rules(rulesList_postPrune)

print('\n')
print('Post-prune:')
print('Accuracy: ' + str(accuracy_score(y_test, Y_pred)))
print('Precision: ' + str(precision_score(y_test, Y_pred, average = None, zero_division = 0)[1]))
print('Recall: ' + str(recall_score(y_test, Y_pred, average = None, zero_division=0)[1]))
print('f1: ' + str(f1_score(y_test, Y_pred, pos_label='Positive')))
print('\n')





Pre-prune: 
Accuracy: 0.5192307692307693
Precision: 0.5818181818181818
Recall: 0.5423728813559322
f1: 0.5614035087719298


Prune process:

last accuracy: 0.9230769230769231 //// current accuracy: 0.9326923076923077
last accuracy: 0.9230769230769231 //// current accuracy: 0.9326923076923077
last accuracy: 0.9230769230769231 //// current accuracy: 0.9326923076923077
last accuracy: 0.9230769230769231 //// current accuracy: 0.9326923076923077
last accuracy: 0.9230769230769231 //// current accuracy: 0.9326923076923077
last accuracy: 0.9230769230769231 //// current accuracy: 0.9230769230769231


Post-prune:
Accuracy: 0.5288461538461539
Precision: 0.5862068965517241
Recall: 0.576271186440678
f1: 0.5811965811965812




<center><h2> Implementation Details </center></h2>

We used 2 classes to impelement decision tree structure from scratch. Their names are Node and DecisionTreeClassifier.

### Node Class

Node class is to classify the nodes of the tree like the name suggests. It stores these inside:

__feature_index__: Index of the feature that we used to split the data for that node.

__feature_type__: Type of that feature.(Discrete or continuous)

__threshold__: Value that we used as a threshold to split the data. 

__left__: Left subtree of the node. For continuous features left subtree has values that are less than or equal to threshold(values <= 42 etc.), for discrete features it has same values as threshold(values == Yes etc.).

__right__: Left subtree of the node. For continuous features left subtree has values that are bigger than threshold(values > 42 etc.), for discrete features it has same values as threshold(values != Yes etc.).

__value__: If the node is a leaf node, it keeps the value(Positive, negative etc.) of it.

### DecisionTreeClassifier

Build the tree and do predictions. It also stores the root of the tree.

__build_tree__: Builds the tree recursively based on the dataset. It goes left to right.(first builds left subtree) Uses purity as a stop sign.

__get_best_split__: Finds the best split we can do based on ID3 algorithm. To find the best split first it loops over all the features.Then finds possible thresholds for that feature and type of that feature. If the feature type is continuous we loop over the possible thresholds, find their information gain and store those with needed informations about that feature in a lis. Else if the feature type is discrete we check if yes is in the thresholds or not. This shows us if the feature is gender or not. Again store their information gains and infos in the list. After all that it returns a dictionary with the best split and its informations(need those to create nodes) in it.

__is_pure__: If any value of the feature represents only one class we call that a pure feature. In this function we check if the class feature(the feature we do the classification with) is pure or not. This is needed while building the decision tree because if the class is pure we cannot split the data anymore.

__split_cont__: Splits the data into two parts based on continuous feature. Dataset_left has values that are less than or equal to threshold value and dataset_right has values that are bigger than the threshold.

__split_disc__: Splits the data into two parts based on discrete feature. Dataset_left has values that are same with threshold value and dataset_right has values that are different than the threshold.

__entropy__: Get the number of positive and negative samples of whole dataset. Apply entropy formula, return the result.

__Information_gain__: Finds the information gain from a certain split. Information gain is the difference between entropy before the split and entropy after the split.

__get_featureType__: Returns the type of the feature. Either discrete or continuous.

__fit__: Fits the training data into tree.

__predict__: Predicts the class of the test set. 

__make_prediction__: Recursively travels root the decision tree based on feature values for a test row and makes a prediction for it.

### Implementation Details of Prune Operation

__twig_check__: Takes one parameter node. Checks children of this node if both Positive or negative returns true else returns false.

__detect_min_recursive__: Takes two parameter a node and a list. The node supposed to be root of the tree, traverses tree, appends information_gain of the node if the current node is a twig.

__detect_min_infoGain__: Creates an empty list, calls detect_min_recursive() with this list and returns the min info_gain value in the list.

__find_majority__: Takes a list as parameter which is created in get_node_recursive method, this list contains some feature_index and their values, in order to determine which rows should be considered while calculating the majority.

__get_node_recursive__: Parameters are given_node(which is supposed to be root of the tree), info_gain(information gain of the node that we are searching for), val_data_x, val_data_y, return_data, road_data. 

Traverses the tree starting with root(the given node), if finds the root with given info_gain calls find_majority() method, converts this node to a leaf with majority as its value. Using val_data_x, and val_data_y (the parameters) calculates the accuracy again if new accuracy is bigger appends "True" into the return data, else appends "False" into the return_data. Through out the searching process saves the informations of the node into the road_data list. This list is used to calculate majority.

__prune__: The main method to execute prune process. Takes val_data_x, val_data_y as parameter. Calls get_node_recursive by using this two parameter. Executes operation inside a while loop until get_node_recursive returns False.

### Helper Functions For Converting Tree To Rules

__detect_ways__: Traverses the tree, every time reaches to a Leaf saves the way. Returns a list of ways to Leafs.

__print_rules__: Calls detect_ways(), takes the list returned from detect_ways, unpacks it and prints rules.

__print_givenList__: Just a simple function, takes a list as parameter and prints it.

# Error Analysis For Classification

| Fold | Accuracy | Precision | Recall | f1_score |
| :- | :-: | :-: | :-: | :-: |
| Fold 1 | 0.95192 | 0.95522 | 0.96969 | 0.96240 | 
| Fold 2 | 0.98076 | 0.97014 | __1.00000__ | 0.98484 |
| Fold 3 | 0.94230 | 0.96610 | 0.93442 | 0.95000 |
| Fold 4 | __0.99038__ | __1.00000__ | 0.98550 | __0.99270__ |
| Fold 5 | 0.92307 | 0.98113 | 0.88135 | 0.92857 |



#### For Accuracy

Highest accuracy value is in Fold 4. Highest accuracy is: 0.9903846153846154

#### For Precision

Highest precision value is in Fold 4. Highest Precision is: 1.0

#### For Recall

Highest Recall value is in Fold 2. Highest recall is: 1.0

#### For f1

Highest f1 value is in Fold4. Highest f1 is: 0.9927007299270074

### Conclusion

Best performing fold is Fold 4.

### Rules for tree when test fold is fold 4:

Polyuria=Yes^Polydipsia=Yes => ['Positive']

Polyuria=Yes^Polydipsia=No^Itching=Yes^Genital thrush=Yes^Obesity=Yes^sudden weight loss=Yes => ['Negative']

Polyuria=Yes^Polydipsia=No^Itching=Yes^Genital thrush=Yes^Obesity=Yes^sudden weight loss=No => ['Positive']

Polyuria=Yes^Polydipsia=No^Itching=Yes^Genital thrush=Yes^Obesity=No => ['Positive']

Polyuria=Yes^Polydipsia=No^Itching=Yes^Genital thrush=No^Age<57=Yes => ['Positive']

Polyuria=Yes^Polydipsia=No^Itching=Yes^Genital thrush=No^Age<57=No => ['Negative']

Polyuria=Yes^Polydipsia=No^Itching=No => ['Positive']

Polyuria=No^Gender=Yes^Polydipsia=Yes^Irritability=Yes => ['Positive']

Polyuria=No^Gender=Yes^Polydipsia=Yes^Irritability=No^muscle stiffness=Yes^visual blurring=Yes => ['Negative']

Polyuria=No^Gender=Yes^Polydipsia=Yes^Irritability=No^muscle stiffness=Yes^visual blurring=No => ['Positive']

Polyuria=No^Gender=Yes^Polydipsia=Yes^Irritability=No^muscle stiffness=No^Age<47=Yes => ['Positive']

Polyuria=No^Gender=Yes^Polydipsia=Yes^Irritability=No^muscle stiffness=No^Age<47=No^sudden weight loss=Yes => ['Positive']

Polyuria=No^Gender=Yes^Polydipsia=Yes^Irritability=No^muscle stiffness=No^Age<47=No^sudden weight loss=No => ['Negative']

Polyuria=No^Gender=Yes^Polydipsia=No^Irritability=Yes^Genital thrush=Yes => ['Positive']

Polyuria=No^Gender=Yes^Polydipsia=No^Irritability=Yes^Genital thrush=No^Age<42=Yes^Polyphagia=Yes => ['Positive']

Polyuria=No^Gender=Yes^Polydipsia=No^Irritability=Yes^Genital thrush=No^Age<42=Yes^Polyphagia=No => ['Negative']

Polyuria=No^Gender=Yes^Polydipsia=No^Irritability=Yes^Genital thrush=No^Age<42=No => ['Negative']

Polyuria=No^Gender=Yes^Polydipsia=No^Irritability=No^weakness=Yes^Itching=Yes^Alopecia=Yes => ['Negative']

Polyuria=No^Gender=Yes^Polydipsia=No^Irritability=No^weakness=Yes^Itching=Yes^Alopecia=No^sudden weight loss=Yes => ['Positive']

Polyuria=No^Gender=Yes^Polydipsia=No^Irritability=No^weakness=Yes^Itching=Yes^Alopecia=No^sudden weight loss=No => ['Negative']

Polyuria=No^Gender=Yes^Polydipsia=No^Irritability=No^weakness=Yes^Itching=No^Alopecia=Yes^sudden weight loss=Yes => ['Negative']

Polyuria=No^Gender=Yes^Polydipsia=No^Irritability=No^weakness=Yes^Itching=No^Alopecia=Yes^sudden weight loss=No => ['Positive']

Polyuria=No^Gender=Yes^Polydipsia=No^Irritability=No^weakness=Yes^Itching=No^Alopecia=No => ['Negative']

Polyuria=No^Gender=Yes^Polydipsia=No^Irritability=No^weakness=No^partial paresis=Yes => ['Positive']

Polyuria=No^Gender=Yes^Polydipsia=No^Irritability=No^weakness=No^partial paresis=No => ['Negative']

Polyuria=No^Gender=No^Alopecia=Yes^delayed healing=Yes => ['Negative']

Polyuria=No^Gender=No^Alopecia=Yes^delayed healing=No => ['Positive']

Polyuria=No^Gender=No^Alopecia=No^Age<36=Yes^muscle stiffness=Yes => ['Positive']

Polyuria=No^Gender=No^Alopecia=No^Age<36=Yes^muscle stiffness=No^Age<34=Yes => ['Negative']

Polyuria=No^Gender=No^Alopecia=No^Age<36=Yes^muscle stiffness=No^Age<34=No^Irritability=Yes => ['Negative']

Polyuria=No^Gender=No^Alopecia=No^Age<36=Yes^muscle stiffness=No^Age<34=No^Irritability=No => ['Positive']

Polyuria=No^Gender=No^Alopecia=No^Age<36=No => ['Positive']

#### Computation Time

Computation time: 1.1788227558135986

Not too long or slow but we can say that, as the branch of three increases, computation time also increases.

 __Misclassified samples from Fold 5__

[42, 'Male', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No', 'Yes', 'No', 'No', 'Yes', 'No', 'No']
Expected: Positive Predicted: Negative

[54, 'Male', 'No', 'No', 'No', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'Yes']
Expected: Positive Predicted: Negative

These two misclassified because 

## Analysis for Part 2: Prune



| Pre/Post | Accuracy | Precision | Recall | f1_score |
| :- | :-: | :-: | :-: | :-: |
| __Pre_prune__ | 0.51923 | 0.58181 | 0.54237 | 0.56140 | 
| __Post_prune__ | 0.50961 | 0.57407 | 0.52542 | 0.54867 |


We have executed prune operation with respect to validation data. The accuracy on validation data was increased from __0.923__ to __0.932__ but the accuracy on test set was decreased from __0.500__ to __0.490__. Also precision, recall and f1 scores were decreased. Before prune operation the tree was more spesific, some twigs were checking for data yes or no and resulting as Positive or Negative. But these twigs are generalized in prune process so we can say that prune operation reduces overfitting and this is also compatible with our pre-prune and post-prune data.

### Rules of Tree Before Prune Operation

Polydipsia=Yes^Polyuria=Yes => ['Positive']

Polydipsia=Yes^Polyuria=No^muscle stiffness=Yes^Gender=Yes => ['Negative']

Polydipsia=Yes^Polyuria=No^muscle stiffness=Yes^Gender=No => ['Positive']

Polydipsia=Yes^Polyuria=No^muscle stiffness=No => ['Positive']

Polydipsia=No^Polyuria=Yes^Age<66=Yes^Obesity=Yes^Age<44=Yes => ['Negative']

Polydipsia=No^Polyuria=Yes^Age<66=Yes^Obesity=Yes^Age<44=No => ['Positive']

Polydipsia=No^Polyuria=Yes^Age<66=Yes^Obesity=No => ['Positive']

Polydipsia=No^Polyuria=Yes^Age<66=No^Gender=Yes => ['Negative']

Polydipsia=No^Polyuria=Yes^Age<66=No^Gender=No => ['Positive']

Polydipsia=No^Polyuria=No^Gender=Yes^Irritability=Yes^Age<43=Yes^Age<40=Yes => ['Negative']

Polydipsia=No^Polyuria=No^Gender=Yes^Irritability=Yes^Age<43=Yes^Age<40=No => ['Positive']

Polydipsia=No^Polyuria=No^Gender=Yes^Irritability=Yes^Age<43=No => ['Negative']

Polydipsia=No^Polyuria=No^Gender=Yes^Irritability=No^partial paresis=Yes^muscle stiffness=Yes => ['Negative']

Polydipsia=No^Polyuria=No^Gender=Yes^Irritability=No^partial paresis=Yes^muscle stiffness=No => ['Positive']

Polydipsia=No^Polyuria=No^Gender=Yes^Irritability=No^partial paresis=No => ['Negative']

Polydipsia=No^Polyuria=No^Gender=No^Alopecia=Yes => ['Negative']

Polydipsia=No^Polyuria=No^Gender=No^Alopecia=No^Age<36=Yes^visual blurring=Yes => ['Positive']

Polydipsia=No^Polyuria=No^Gender=No^Alopecia=No^Age<36=Yes^visual blurring=No^Age<34=Yes => ['Negative']

Polydipsia=No^Polyuria=No^Gender=No^Alopecia=No^Age<36=Yes^visual blurring=No^Age<34=No^weakness=Yes => ['Negative']

Polydipsia=No^Polyuria=No^Gender=No^Alopecia=No^Age<36=Yes^visual blurring=No^Age<34=No^weakness=No => ['Positive']

Polydipsia=No^Polyuria=No^Gender=No^Alopecia=No^Age<36=No => ['Positive']

### Rules of Tree After Prune Operation

Polydipsia=Yes^Polyuria=Yes => ['Positive']

Polydipsia=Yes^Polyuria=No^muscle stiffness=Yes^Gender=Yes => ['Negative']

Polydipsia=Yes^Polyuria=No^muscle stiffness=Yes^Gender=No => ['Positive']

Polydipsia=Yes^Polyuria=No^muscle stiffness=No => ['Positive']

Polydipsia=No^Polyuria=Yes^Age<66=Yes^Obesity=Yes^Age<44=Yes => ['Negative']

Polydipsia=No^Polyuria=Yes^Age<66=Yes^Obesity=Yes^Age<44=No => ['Positive']

Polydipsia=No^Polyuria=Yes^Age<66=Yes^Obesity=No => ['Positive']

Polydipsia=No^Polyuria=Yes^Age<66=No => ['Positive']

Polydipsia=No^Polyuria=No^Gender=Yes^Irritability=Yes^Age<43=Yes^Age<40=Yes => ['Negative']

Polydipsia=No^Polyuria=No^Gender=Yes^Irritability=Yes^Age<43=Yes^Age<40=No => ['Positive']

Polydipsia=No^Polyuria=No^Gender=Yes^Irritability=Yes^Age<43=No => ['Negative']

Polydipsia=No^Polyuria=No^Gender=Yes^Irritability=No => ['Negative']

Polydipsia=No^Polyuria=No^Gender=No^Alopecia=Yes => ['Negative']

Polydipsia=No^Polyuria=No^Gender=No^Alopecia=No^Age<36=Yes^visual blurring=Yes => ['Positive']

Polydipsia=No^Polyuria=No^Gender=No^Alopecia=No^Age<36=Yes^visual blurring=No => ['Negative']

Polydipsia=No^Polyuria=No^Gender=No^Alopecia=No^Age<36=No => ['Positive']